In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout, Flatten,BatchNormalization, GaussianNoise
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [2]:
#reading in the diagnosis data
m = pd.read_csv("ground_truth.csv").drop("Unnamed: 0", axis=1).rename(columns = {"Subject":"subject", "Group": "label"})

In [3]:
#making sure all the diagnosis agree
m = m[m["label"] != -1]

In [4]:
#merging SNPs with diagnosis
temp_vcf = pd.read_pickle("all_vcfs.pkl")
total_rows=temp_vcf.shape[0]
vcf=pd.DataFrame()
chunksize=5
for i in range(total_rows//chunksize):
    print(i)
    vcf_chunk=temp_vcf.iloc[(i*chunksize):((i+1)*chunksize)]
    vcf_chunk=vcf_chunk.merge(m[["subject", "label"]], on = "subject")
    if(i==0):
        vcf=vcf_chunk
        continue
    vcf=vcf.append(vcf_chunk)
    # #breaking after considering 1000 rows as its taking too long to process
    # if(i==2):
    #     break

# vcf = chunk.merge(m[["subject", "label"]], on = "subject")
# vcf

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


KeyboardInterrupt: 

In [ ]:
vcf = vcf.drop_duplicates()
print(vcf)

       subject  Group  11103  11104  11105  11106  11107  11108  11109  11110  \
0   002_S_0413    0.0    2.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3   002_S_0685    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
6   002_S_0729    2.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9   002_S_1155    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
13  002_S_1268    2.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0   002_S_1280    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4   002_S_2010    0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
6   002_S_2043    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
8   002_S_2073    0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9   002_S_4171    2.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0   002_S_4213    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2   002_S_4219    1.0    0.0

In [ ]:
#reading in the overlap test set
# ts = pd.read_csv("overlap_test_set.csv")
# ts = pd.read_csv("ground_truth.csv").rename(columns={'Subject':'subject'})

#removing ids from the overlap test set, saving it as a new variable
# vcf1 = vcf[vcf["subject"].isin(list(ts["subject"].values))]
vcf1 = vcf
vcf1=vcf1.fillna(0)
vcf1=vcf1.replace(r'[^0-9]',3,regex=True)

In [ ]:
# Using Random Forest to reduce feature dimensions
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))

In [ ]:
cols = list(set(vcf1.columns) - set(["subject", "Group", "label"]))
X = vcf1[cols].values.astype(int)
# y = vcf1["label"].astype(int).values
y = vcf1["Group"].astype(int).values
columns_list=vcf1[cols].columns
print(X)
print(y)
for i in range(len(y)):
    y[i] = y[i]-1

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[0 0 2 1 2 0 0 1 0 2 0 1 2 1 1]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [ ]:
#fitting random forest only on the training data so that there is not influence on the testing performance
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [ ]:
# selected_feat= X_train.columns[(sel.get_support())]
selected_feat= columns_list[(sel.get_support())]
len(selected_feat)

350

In [ ]:
l = ["label", "subject", "Group"]
l.extend(selected_feat)

In [ ]:
#saving the features with the old dataframe so that the overlap test set can still be used when combining all data
vcf[l].to_pickle("vcf_select.pkl")

In [ ]:
# X_train.to_pickle("X_train_vcf.pkl")
# y_train.to_pickle("y_train_vcf.pkl")

# X_test.to_pickle("X_test_vcf.pkl")
# y_test.to_pickle("y_test_vcf.pkl")
with open("X_train_vcf.pkl",'wb') as f:
    np.save(f,X_train)
with open("y_train_vcf.pkl",'wb') as f:
    np.save(f,y_train)
with open("X_test_vcf.pkl",'wb') as f:
    np.save(f,X_test)
with open("y_test_vcf.pkl",'wb') as f:
    np.save(f,y_test)   